In [4]:
from google.colab import files
uploaded = files.upload()


Saving fear_greed_index.csv to fear_greed_index.csv
Saving historical_data.csv to historical_data.csv


In [7]:
import pandas as pd
import plotly.express as px
import io

In [8]:
trades=pd.read_csv(io.BytesIO(uploaded["historical_data.csv"]))


In [9]:
fg = pd.read_csv(io.BytesIO(uploaded["fear_greed_index.csv"]))



In [10]:
trades['Timestamp IST'] = pd.to_datetime(
    trades['Timestamp IST'],
    format="%d-%m-%Y %H:%M",
    errors='coerce'
)
trades['date'] = trades['Timestamp IST'].dt.date

In [11]:
trades['Closed PnL'] = pd.to_numeric(trades['Closed PnL'], errors='coerce')

fg['date'] = pd.to_datetime(fg['date']).dt.date
fg = fg.rename(columns={'classification': 'sentiment'})

merged = trades.merge(
    fg[['date', 'sentiment', 'value']],
    on='date',
    how='left'
)

# 6. Create helper features
merged['is_win'] = (merged['Closed PnL'] > 0).astype(int)
merged['side_lower'] = merged['Side'].astype(str).str.lower()
merged['is_buy'] = (merged['side_lower'] == 'buy').astype(int)
merged['is_sell'] = (merged['side_lower'] == 'sell').astype(int)

In [12]:
sentiment_summary = (
    merged
    .groupby('sentiment')
    .agg(
        total_trades=('Account', 'count'),
        total_pnl=('Closed PnL', 'sum'),
        avg_pnl_per_trade=('Closed PnL', 'mean'),
        win_rate=('is_win', 'mean'),
        avg_size_tokens=('Size Tokens', 'mean'),
        avg_size_usd=('Size USD', 'mean'),
        buy_share=('is_buy', 'mean'),
        sell_share=('is_sell', 'mean'),
        avg_fgi_value=('value', 'mean')
    )
    .reset_index()
)
sentiment_summary['win_rate_percent'] = sentiment_summary['win_rate'] * 100


print(sentiment_summary)


       sentiment  total_trades     total_pnl  avg_pnl_per_trade  win_rate  \
0   Extreme Fear         21400  7.391102e+05          34.537862  0.370607   
1  Extreme Greed         39992  2.715171e+06          67.892861  0.464943   
2           Fear         61837  3.357155e+06          54.290400  0.420768   
3          Greed         50303  2.150129e+06          42.743559  0.384828   
4        Neutral         37686  1.292921e+06          34.307718  0.396991   

   avg_size_tokens  avg_size_usd  buy_share  sell_share  avg_fgi_value  \
0      1675.893319   5349.731843   0.510981    0.489019      19.690514   
1      8832.415746   3112.251565   0.448590    0.551410      79.552986   
2      2992.641845   7816.109931   0.489513    0.510487      33.202549   
3      4715.327756   5736.884375   0.488559    0.511441      67.977775   
4      4383.368714   4782.732661   0.503343    0.496657      48.661174   

   win_rate_percent  
0         37.060748  
1         46.494299  
2         42.076750  
3   

In [13]:
account_sent = (
    merged
    .groupby(['Account', 'sentiment'])
    .agg(
        trades=('Coin', 'count'),
        total_pnl=('Closed PnL', 'sum'),
        avg_pnl_per_trade=('Closed PnL', 'mean'),
        win_rate=('is_win', 'mean'),
        avg_size_tokens=('Size Tokens', 'mean'),
        avg_size_usd=('Size USD', 'mean'),
        buy_share=('is_buy', 'mean'),
        sell_share=('is_sell', 'mean')
    )
    .reset_index()
)

account_perf_by_sent = (
    account_sent
    .assign(is_profitable=lambda x: x['total_pnl'] > 0)
    .groupby(['sentiment', 'is_profitable'])
    .size()
    .reset_index(name='num_accounts')
)

total_accounts_by_sent = account_perf_by_sent.groupby('sentiment')['num_accounts'] \
                                             .sum().reset_index(name='total_accounts')

merged_acc = account_perf_by_sent.merge(total_accounts_by_sent, on='sentiment')
merged_acc['profitable_share'] = merged_acc['num_accounts'] / merged_acc['total_accounts']

print(merged_acc)


       sentiment  is_profitable  num_accounts  total_accounts  \
0   Extreme Fear          False            11              32   
1   Extreme Fear           True            21              32   
2  Extreme Greed          False             4              30   
3  Extreme Greed           True            26              30   
4           Fear          False             6              32   
5           Fear           True            26              32   
6          Greed          False             4              31   
7          Greed           True            27              31   
8        Neutral          False            10              31   
9        Neutral           True            21              31   

   profitable_share  
0          0.343750  
1          0.656250  
2          0.133333  
3          0.866667  
4          0.187500  
5          0.812500  
6          0.129032  
7          0.870968  
8          0.322581  
9          0.677419  


In [14]:

fig = px.bar(
    sentiment_summary,
    x="sentiment",
    y="avg_pnl_per_trade",
    title="Average PnL per Trade by Sentiment",
    labels={"sentiment": "Sentiment", "avg_pnl_per_trade": "Avg PnL per Trade"},
)
fig.show()

fig = px.bar(
    sentiment_summary,
    x="sentiment",
    y="win_rate_percent",
    title="Win Rate by Sentiment (%)",
    labels={"sentiment": "Sentiment", "win_rate_percent": "Win Rate (%)"}
)
fig.show()
